This notebook explore the publications citing Galaxy

In [1]:
import datetime
import matplotlib.pyplot as plt
import os
import pandas as pd
import pprint
import re
import requests
import time

from pathlib import Path
from pyzotero import zotero
from scholarly import ProxyGenerator, scholarly

In [2]:
result_dp = Path("../results/citations/")
result_dp.mkdir(parents=True, exist_ok=True)

In [3]:
searched_tags = [
    "bacteri", # bacteria, bacterio
    "prokaryot", # prokaryote, prokaryotic
    "microb", # microbiome, microbiota, microbial, microbiology, microbe
    "pathogen",
    "virus",
    "phage",
    "archae",
    "flora",
    "microecology",
    "microorganism", "micro-organism",
    "microbiome" "microbiota",
    "metabarcod", # metabarcode, metabarcoding
        "16s", "16 s",
        "18s",
        "amplicon",
        "metataxonom",
    "metagenom", # metagenome, metagenomic
    "metatranscriptom", # metatranscriptome, metatranscriptomic
    "metaproteom", # metaproteome, metaproteomic
    "multi-locus sequence typing", "multilocus sequence typing", "mlst",
    "otu",
]

In [4]:
def create_grouping_keywords():
    return {
        'Targeted organisms': {
            'Bacteria': {
                'keywords': ['bacteri', 'prokaryot'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Virus': {
                'keywords': ['virus', 'phage'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Archaea': {
                'keywords': ['archae'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Eukaryote': {
                'keywords': ['eukaryot'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Microbiome': {
                'keywords': ['microbiome', 'microbiota', 'flora', 'ecology', 'community', 'communities'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Pathogen': {
                'keywords': ['pathogen'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            }
        },
        'Technical target': {
            'Isolate': {
                'keywords': ['isolate'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Community (taxonomy) profiling': {
                'keywords': ['otu', 'asv', 'taxonom', 'community', 'communities', 'ecosystem', 'diversity', 'ecology'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Functional analysis': {
                'keywords': ['function'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Interactome': {
                'keywords': ['interaction'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'AMR': {
                'keywords': ['antimicrobial', 'resistan', 'amr'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'MAGs': {
                'keywords': ['mags', 'metagenome-assembled genome', 'assembly'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Gene identification / Biomarker': {
                'keywords': ['gene identification', 'biomarker'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'SNP': {
                'keywords': ['snp', 'single-nucleotide polymorphism'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            '(M)LST': {
                'keywords': ['mlst', 'locus sequence typing'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Annotation': {
                'keywords': ['annotation'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Variant': {
                'keywords': ['variant'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Comparative analysis': {
                'keywords': ['comparative'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
        },
        'Methods': {
            'Metabarcoding': {
                'keywords': ['metabarcod', '16s', '16 s', '18s', 'amplicon', 'metataxonom'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            '(Meta)genomics': {
                'keywords': ['genom'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Metagenomics': {
                'keywords': ['metagenom'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            '(Meta)transcriptomics': {
                'keywords': ['transcriptom'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Metatranscriptomics': {
                'keywords': ['metatranscriptom'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            '(Meta)proteomics': {
                'keywords': ['proteom', 'mass spectrometry'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Metaproteomics': {
                'keywords': ['metaproteom'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Metabolomics': {
                'keywords': ['metabolom', 'metabolite'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Imaging': {
                'keywords': ['imaging'],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
        },
        'Topics': {
            'Ecosystem Dynamics & Biodiversity': {
                'keywords': [
                    'environment',
                    'diversity',
                    'ecosystem',
                    'plant',
                    'corrosion',
                    'geograph',
                    'compost',
                    'fish',
                    'soil',
                    'biofilm',
                    'water',
                    'aquaculture',
                    'ecolog',
                    'ferment',
                    'ocean',
                    'marine',
                    'decomp',
                    'biofuel',
                    'permafrost',
                    'microbial mat',
                    'rhizosphere',
                    'grassland',
                    'biodegrada',
                    'biogeochem',
                    'succession',
                    'trophic',
                    'food web',
                    'community assembly',
                    'resilience',
                    'stability',
                    'niche',
                    'alpha diversity',
                    'beta diversity',
                    'symbiosis',
                    'competition',
                    'functional diversity',
                    'phylogen',
                    'river',
                    'crop',
                    'bird',
                    'cyanobacteri',
                    'aquatic',
                ],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
            'Health & Disease': {
                'keywords': [
                    'health',
                    'disease',
                    'pathogen',
                    'antibiotic',
                    'epidemio',
                    'clinic',
                    'syndrom',
                    'antimicrob',
                    'patient',
                    'cancer',
                    'covid',
                    'hiv',
                    'diagnostic',
                    'food safety',
                    'toxic',
                    'infest',
                    'infect',
                    'prebiotic',
                    'probiotic',
                    'obesity',
                    'immun',
                    'food safety',
                    'gut microbiota',
                    'dysbiosis',
                    'host-microbe',
                    'immunomodulation',
                    'inflammation',
                    'gut-brain axis',
                    'metabolic syndrome',
                    'FMT', 
                    'SCFA',
                    'healthcare',
                    'virulence',
                    'drug',
                ],
                'location': ['tag', 'title', 'abstract'],
                'counts': 0
            },
        },
    }

def find_kw(string, kws):
    '''
    Return if a 1 keyword in a list is found in a string

    :param string: string to search
    :param kws: list of keywords to search
    '''
    found = False
    for kw in kws:
        if kw in string:
            found = True
    return found

# Galaxy Publication Library

The Galaxy Project uses the Zotero social bookmarking service to organize publications that reference Galaxy. The Galaxy Group lists published articles, conference proceedings, theses, book chapters and books that use, extend, reference or implement Galaxy. 

In [5]:
ZOTERO_API = os.getenv('ZOTERO_API')

In [6]:
zot = zotero.Zotero('1732893', 'group', ZOTERO_API)

In this Galaxy Publication Library, the number of elements is:

In [7]:
element_nb = zot.count_items()
element_nb

23188

## Extract items based on keywords in tags, titles, and abstracts

The library makes extensive use of tags to categorize publications and enable smarter searching. But only papers added after the library was moved to Zotero in September 2017 have automatically added tags.

We extract items related to microbial data analysis by checking if their title, abstract, or tags contains any of following keywords:

In [8]:
conserved_items = []
conserved_tags = set()
excluded_tags = set()
years = {
    "all": {},
    "microbial": {}
}


for y in range(2005, datetime.date.today().year+2):
    years["all"].setdefault(f"{y}", 0)
    years["microbial"].setdefault(f"{y}", 0)

for item in zot.everything(zot.top()):
    year = 0
    
    # get year
    if "date" in item["data"]:
        match = re.match(r".*(2[0-9]{3})", item["data"]["date"])
        if match is not None:
            year = match.group(1)
            years["all"][year] += 1
    
    to_add = False

    # search in tags
    for tag in item['data']['tags']:
        for st in searched_tags:
            if st in tag['tag'].lower():
                conserved_tags.add(tag['tag'].lower())
                to_add = True
            else:
                excluded_tags.add(tag['tag'].lower())

    # search in title and abstract
    for st in searched_tags:
        if 'title' in item['data'] and st in item['data']['title'].lower():
            to_add = True
        if 'abstractNote' in item['data'] and st in item['data']['abstractNote'].lower():
            to_add = True

    # keep item if found
    if to_add:
        conserved_items.append(item)
        if year != 0:
            years["microbial"][year] += 1

Number and percentage of publications related to microbioal data analysis

In [9]:
conserved_element_nb = len(conserved_items)
print(f'{conserved_element_nb} {100 * conserved_element_nb/element_nb}')

4896 21.114369501466275


## Explore conserved items

### Per year

In [10]:
year_df = pd.DataFrame.from_dict(years)
year_df

all  microbial
2005     1          0
2006     4          0
2007    12          0
2008    32          1
2009    53          3
2010   107         10
2011   205         24
2012   398         73
2013   506         97
2014   741        149
2015   929        243
2016  1125        281
2017  1334        397
2018  1576        601
2019  1932        874
2020  2165       1124
2021   665        341
2022   102         44
2023   474        207
2024   557        255
2025   343        154
2026     0          0

In [11]:
fig, ax = plt.subplots()
fig.set_dpi(300)
(year_df
     .plot.bar(ax=ax, colormap='Paired'))
plt.ylabel('Publications in Galaxy Publication Library')

Text(0, 0.5, 'Publications in Galaxy Publication Library')

### Per keywords

We group the extracted publications by pre-defined groups given keywords in tags, title, and/or abstract.

In [12]:
not_grouped = []
grouping_keywords = create_grouping_keywords()
for item in conserved_items:
    in_group = False
    for g, group in grouping_keywords.items():
        for sg, subgroup in group.items():
            found = False
            if 'tag' in subgroup['location']:
                for tag in item['data']['tags']:
                    found |= find_kw(tag['tag'].lower(), subgroup['keywords'])
            if 'title' in subgroup['location'] and 'title' in item['data']:
                found |= find_kw(item['data']['title'].lower(), subgroup['keywords'])
            if 'abstract' in subgroup['location'] and 'abstractNote' in item['data']:
                found |= find_kw(item['data']['abstractNote'].lower(), subgroup['keywords'])
            subgroup['counts'] += found
            in_group |= found

    if not in_group:
        not_grouped.append(item)

Number of elements not grouped

In [13]:
f'{len(not_grouped)} / {len(conserved_items)}'

'6 / 4896'

In [14]:
group_dfs = {}
for g, group in grouping_keywords.items():
    group_dfs[g] = (pd.DataFrame.from_dict(group, orient='index')
                    .assign(percentage=lambda df: 100*df.counts/len(conserved_items)))

In [ ]:
def plot_kw_percentage(group, dfs, ax):
    '''
    Plot percentage of publications with keywords related a group

    :param group: group
    :param dfs: dictionary with dataframes
    :param ax: axis
    '''    
    (dfs[group].percentage
         .sort_values()
         .plot.bar(ax=ax, colormap='Paired'))
    #plt.xticks(rotation=45, ha='right')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
    ax.set_title(group, fontsize=20)

fig, axs = plt.subplots(1, 4, sharey=True, figsize=(20, 10))
fig.set_dpi(300)
for i, group in enumerate(group_dfs):
    plot_kw_percentage(group, group_dfs, axs[i])
axs[0].set_ylabel('% in the microbial related publications from Galaxy Publication Library', fontsize=15)

Text(0, 0.5, '% in the microbial related publications from Galaxy Publication Library')

# Google Scholar and Semantic Search

There is now a [Google Scholar profile](https://scholar.google.de/citations?hl=de&user=3tSiRGoAAAAJ) with all major publications of the Galaxy Project. We can use it to extract the citations for the papers using Semantic Search (Google Scholar blocking requests on citations)

In [ ]:
def start_proxy():
    success = pg.FreeProxies()#SingleProxy(http = proxy, https = proxy)
    scholarly.use_proxy(pg)
    
def get_request_json(url: str, headers: dict, retries: int = 10, delay: float = 120) -> dict:
    """
    Perform a GET request to retrieve JSON output from a specified URL, with retry on ConnectionError.

    :param url: URL to send the GET request to.
    :param headers: Headers to include in the GET request.
    :param retries: Number of retry attempts in case of a ConnectionError (default is 3).
    :param delay: Delay in seconds between retries (default is 2.0 seconds).
    :return: JSON response as a dictionary, or None if all retries fail.
    :raises ConnectionError: If all retry attempts fail due to a connection error.
    :raises SystemExit: For any other request-related errors.
    """
    attempt = 0  # Track the number of attempts

    while attempt < retries:
        try:
            r = requests.get(url, auth=None, headers=headers)
            r.raise_for_status()  # Raises an HTTPError for unsuccessful status codes
            return r.json()  # Return JSON response if successful
        except requests.exceptions.HTTPError as e:
            attempt += 1
            if attempt == retries:
                raise ConnectionError(
                    "Connection aborted after multiple retries: Remote end closed connection without response"
                ) from e
            print(f"Connection error on attempt {attempt}/{retries}. Retrying in {delay} seconds...")
            time.sleep(delay)  # Wait before retrying
        except requests.exceptions.RequestException as e:
            # Handles all other exceptions from the requests library
            raise SystemExit(f"Request failed: {e}")
        except ValueError as e:
            # Handles cases where the response isn't valid JSON
            raise ValueError("Response content is not valid JSON") from e

    # Return None if all retries are exhausted and no response is received
    return {}

def find_paper_id(title):
    "From semanticscholar"
    url = f"https://api.semanticscholar.org/graph/v1/paper/search/match?query={title}"
    response = get_request_json(url, {"Accept": "application/json"})
    if "data" in response:
        return response["data"][0]["paperId"]
    else:
        raise ValueError(f"{title} not found")
        
def get_citation_number(paper_id):
    "From semanticscholar"
    url = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=citationCount"
    response = get_request_json(url, {"Accept": "application/json"})
    if "citationCount" in response:
        return response["citationCount"]
    else:
        raise ValueError(f"No citation count found for {paper_id}")
        
def get_citations(paper_id, offset, limit):
    "From semanticscholar"
    url = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations?fields=title,abstract,year&offset={offset}&limit={limit}"
    response = get_request_json(url, {"Accept": "application/json"})
    if "data" in response:
        return response["data"]
    else:
        raise ValueError(f"No citation found for {paper_id} with offset={offset} and limit={limit}")

In [17]:
#pg = ProxyGenerator()
#start_proxy()
#search_query = scholarly.search_author("Galaxy Project")
#author = scholarly.fill(next(search_query))
author = scholarly.search_author_id("3tSiRGoAAAAJ")
author = scholarly.fill(author, sections=['publications'])

In [18]:
citations = []
all_paper_nb = 0
paper_nb = 0

for pub in author["publications"]:
    all_paper_nb += 1
    
    # Keep only majors Galaxy papers
    keep = False
    for k in ["platform", "a comprehensive approach", "a web‐based genome analysis tool for experimentalists"]:
        if k in pub["bib"]["title"]:
            keep = True
    if not keep:
        continue
    
    paper_nb += 1
        
    print(pub["bib"]["title"])
    print(pub["bib"]["pub_year"])
        
    # Search citations using SemanticScholar
    paper_id = find_paper_id(pub["bib"]["title"])
    citation_number = get_citation_number(paper_id)
    print(f"Citations: {citation_number}")
    
    limit = 1000
    for n in range(0, citation_number, limit):
        citations.extend(get_citations(paper_id, n, limit))
        time.sleep(30)
    time.sleep(60)
    print()

Galaxy: a comprehensive approach for supporting accessible, reproducible, and transparent computational research in the life sciences
2010
Connection error on attempt 1/10. Retrying in 120 seconds...
Connection error on attempt 2/10. Retrying in 120 seconds...
Connection error on attempt 3/10. Retrying in 120 seconds...
Citations: 3648

The Galaxy platform for accessible, reproducible and collaborative biomedical analyses: 2018 update
2018
Citations: 1150

Galaxy: a platform for interactive large-scale genome analysis
2005
Connection error on attempt 1/10. Retrying in 120 seconds...
Connection error on attempt 2/10. Retrying in 120 seconds...
Connection error on attempt 1/10. Retrying in 120 seconds...
Citations: 2125

The Galaxy platform for accessible, reproducible and collaborative biomedical analyses: 2016 update
2016
Citations: 3616

Galaxy: a web‐based genome analysis tool for experimentalists
2010
Citations: 1561

The Galaxy platform for accessible, reproducible and collaborativ

Number of Galaxy papers

In [19]:
all_paper_nb

54

Number of major Galaxy papers

In [20]:
paper_nb

8

Number of publications citing Galaxy (before duplication removal)

In [21]:
len(citations)

13344

In [22]:
grouping_keywords = create_grouping_keywords()

titles = []
all_citing_papers = []
microb_citing_papers = []
for citation in citations:
    if citation["citingPaper"]["title"] in titles:
        continue
    titles.append(citation["citingPaper"]["title"])
    
    paper = {
        "year": int(citation["citingPaper"]["year"]) if citation["citingPaper"]["year"] is not None else 0,
        "title": citation["citingPaper"]["title"],
    }
    abstract = citation["citingPaper"]["abstract"]
    all_citing_papers.append(paper.copy())

    # check if microbio
    microbio = False
    for t in searched_tags:
        if t in paper["title"]:
            microbio = True
        if abstract is not None and t in abstract:
            microbio = True

    if not microbio:
        continue
    
    paper["abstract"] = abstract
    # count keywords
    for g, group in grouping_keywords.items():
        paper[g] = []
        for sg, subgroup in group.items():
            found = False
            if "title" in subgroup["location"]:
                found |= find_kw(paper["title"].lower(), subgroup["keywords"])
            if "abstract" in subgroup["location"] and citation["citingPaper"]["abstract"] is not None:
                found |= find_kw(abstract.lower(), subgroup["keywords"])
            subgroup["counts"] += found
            if found:
                paper[g].append(sg)

    microb_citing_papers.append(paper)        

Number of publications citing Galaxy (after duplication removal)

In [23]:
len(all_citing_papers)

10518

Number of Microbial-related publications citing Galaxy

In [24]:
len(microb_citing_papers)

2903

Percentage of Microbial-related publications citing Galaxy

In [25]:
100*len(microb_citing_papers)/len(all_citing_papers)

27.600304240349878

In [26]:
all_citing_papers_df = pd.DataFrame(all_citing_papers)
all_citing_papers_df.to_csv(result_dp / Path("all_papers.csv"))

In [39]:
microb_papers_df = pd.DataFrame(microb_citing_papers)
for col in grouping_keywords:
    microb_papers_df[col] = microb_papers_df[col].apply(lambda x: ", ".join(str(i) for i in x))
microb_papers_df

year                                              title  \
0     2025  Integrating whole-genome sequencing into antim...   
1     2025  Next-generation sequencing applications in foo...   
2     2025  Comprehensive mapping of transcription termina...   
3     2025  Indirect impact of quorum sensing on corrosion...   
4     2025  ‘What’s in a name? Fit-for-purpose bacterial n...   
...    ...                                                ...   
2898  2024  Nicoliella lavandulae sp. nov., a novel fructo...   
2899  2024  A Novel Bacitracin-like Peptide from Mangrove-...   
2900  2024  Engineering a wolf spider A-family toxin towar...   
2901  2024       Establishing the ELIXIR Microbiome Community   
2902  2025  Dix-seq: An integrated pipeline for fast ampli...   

                                               abstract    Targeted organisms  \
0     SUMMARYAntimicrobial resistance (AMR) poses a ...                         
1     Next-generation sequencing (NGS) has revolutio...  Microbiome, Pathogen   
2     Abstract Recent evidence indicates that the ba...              Bacteria   
3                                                  None              Bacteria   
4     Abstract Rapid and economical DNA sequencing h...  Bacteria, Microbiome   
...                                                 ...                   ...   
2898  Abstract A survey of fructophilic lactic acid ...              Bacteria   
2899  The global rise of antimicrobial resistance (A...              Bacteria   
2900  Peptides with insecticidal, antimicrobial and/...    Bacteria, Pathogen   
2901  Microbiome research has grown substantially ov...            Microbiome   
2902  Rapid advancements in sequencing technologies ...                         

                                       Technical target  \
0                                                   AMR   
1      Community (taxonomy) profiling, Interactome, AMR   
2                                  Comparative analysis   
3                                                         
4                        Community (taxonomy) profiling   
...                                                 ...   
2898                                            Isolate   
2899       Isolate, Community (taxonomy) profiling, AMR   
2900                                                AMR   
2901  Community (taxonomy) profiling, Functional ana...   
2902                     Community (taxonomy) profiling   

                                                Methods  \
0                                        (Meta)genomics   
1     (Meta)genomics, Metagenomics, (Meta)transcript...   
2                 (Meta)genomics, (Meta)transcriptomics   
3                                                         
4           Metabarcoding, (Meta)genomics, Metagenomics   
...                                                 ...   
2898                      Metabarcoding, (Meta)genomics   
2899                       (Meta)genomics, Metabolomics   
2900                                                      
2901  Metabarcoding, (Meta)genomics, Metagenomics, (...   
2902        Metabarcoding, (Meta)genomics, Metagenomics   

                                                 Topics  
0                                      Health & Disease  
1     Ecosystem Dynamics & Biodiversity, Health & Di...  
2                     Ecosystem Dynamics & Biodiversity  
3                     Ecosystem Dynamics & Biodiversity  
4     Ecosystem Dynamics & Biodiversity, Health & Di...  
...                                                 ...  
2898                  Ecosystem Dynamics & Biodiversity  
2899  Ecosystem Dynamics & Biodiversity, Health & Di...  
2900                                   Health & Disease  
2901                  Ecosystem Dynamics & Biodiversity  
2902                  Ecosystem Dynamics & Biodiversity  

[2903 rows x 7 columns]

Number of Microbial-related publications citing Galaxy without Topics

In [28]:
without_topic = microb_papers_df.query('Topics == ""')
len(without_topic)

365

In [47]:
microb_papers_df.loc[microb_papers_df.Topics == "", "Topics"] = ", ".join(grouping_keywords["Topics"].keys())
#microb_papers_df.loc[microb_papers_df.Topics == "", "Topics"] = "Ecosystem Dynamics & Biodiversity"

In [30]:
microb_papers_df.to_csv(result_dp / Path("microbial_papers.csv"))

Numbers in the different *Topics* categories

In [49]:
microb_papers_df["Topics"].value_counts()

Topics
[Ecosystem Dynamics & Biodiversity, Health & Disease]    1112
[Health & Disease]                                        833
[Ecosystem Dynamics & Biodiversity]                       593
[]                                                        365
Name: count, dtype: int64

Numbers in the different *Targeted organisms* categories

In [50]:
microb_papers_df = pd.DataFrame(microb_citing_papers)
targeted_orga = [x for xs in microb_papers_df["Targeted organisms"].to_list() for x in xs]
pd.Series(targeted_orga).value_counts()

Bacteria      1254
Pathogen       887
Microbiome     855
Virus          855
Eukaryote       88
Archaea         74
Name: count, dtype: int64

Numbers in the different *Technical target* categories

In [52]:
microb_papers_df = pd.DataFrame(microb_citing_papers)
targeted_orga = [x for xs in microb_papers_df["Technical target"].to_list() for x in xs]
pd.Series(targeted_orga).value_counts()

Community (taxonomy) profiling     938
Functional analysis                687
Isolate                            535
AMR                                519
Interactome                        345
Variant                            213
Comparative analysis               173
MAGs                               153
Annotation                         136
Gene identification / Biomarker     82
SNP                                 63
(M)LST                              35
Name: count, dtype: int64

Numbers in the different *Methods* categories

In [53]:
microb_papers_df = pd.DataFrame(microb_citing_papers)
targeted_orga = [x for xs in microb_papers_df["Methods"].to_list() for x in xs]
pd.Series(targeted_orga).value_counts()

(Meta)genomics           1404
Metabarcoding             460
(Meta)transcriptomics     354
Metagenomics              305
(Meta)proteomics          131
Metabolomics              125
Metatranscriptomics        39
Metaproteomics             20
Imaging                    15
Name: count, dtype: int64